# fully_CN_Network

## Imports

In [1]:
import tensorflow as tf
import numpy as np
from pprint import pprint
import sys, os

In [6]:
class bn_fully_CN_Network(object):
    """A neural network of multi-path layers.
       Filters for each path have shape_height = 1.
       Parameters cannot be changed after initialzation. Create new network instead."""
    
    def __init__(self,
                 name,
                 wide_filter_widths,
                 width_reduction_factors,
                 dtype = tf.float32,
                 num_freq_channels = 1024,
                 learning_rate = 0.0001,
                 cost_name = 'MSE',
                 threshold = 0.00625,
                 g_shift = 0.01,
                 log_dir = 'logs/',
                 num_1x1_conv_filters = 4):
        
        self.log_dir = log_dir#
        
        self.name = name
        self.wide_filter_widths = wide_filter_widths
        self.width_reduction_factors = width_reduction_factors
        self.dtype = dtype
        self.num_freq_channels = num_freq_channels
        self.learning_rate = learning_rate
        self.cost_name = cost_name
        self.threshold = threshold
        self.g_shift = g_shift
        self.num_1x1_conv_filters = num_1x1_conv_filters
                

    def print_params(self):
        """Prints netwwork parameters"""
        pprint(self._gen_params_dict())
        
    def load_params(self, path):
        """Load in the parameters of an old network, but keep the current network name."""
    
        sys.stdout.write('\rLoading Netowrk Parameters')
        a = load(path + '.npz')
        d = dict(zip(("data1{}".format(k) for k in a), (a[k] for k in a)))
        
        name = self.name
        params = d['data1arr_0'][()]
        for key in params:
            setattr(self, key, params[key])
        self.name = name

    def _gen_params_dict(self):
        d = self.__dict__
        return {key : d[key] for key in d.keys() if key[0] != '_' if 'tensorflow' not in str(type(d[key]))}
        
    def _save_params(self):
        direc = self.log_dir + self.name + '/params/'
        """Not safe - will overwrite existing file."""
        if not os.path.exists(direc):
            os.makedirs(direc)
            
        np.savez(direc + self.__class__.__name__, self._gen_params_dict())   
                
    def create_graph(self):
        self._save_params()
        sys.stdout.write('\rCreating Network Graph')
        
        
        

        def quad_path_layer(input, wide_conv_width, strides, layer_name, num_1x1_conv_filters = 4):

            # convolution filters
            conv_filters = lambda shape : tf.get_variable(name = 'filters',
                                                          dtype = self.dtype,
                                                          shape = shape,
                                                          initializer = tf.contrib.layers.xavier_initializer())


            def _bias_add_scope(input, shape):
                """Creates a scope around a trainable bias and its addition to input"""
                with tf.variable_scope('add_bias'):

                    bias = tf.get_variable(name = 'biases', dtype = self.dtype, shape = shape, initializer = tf.contrib.layers.xavier_initializer())
                    bias_add = tf.nn.bias_add(input, bias)

                return bias_add


            def _conv_scope(input, filter_shape, strides, scope_name = 'convolution'):
                """Creates a scope around a convolution."""
                with tf.variable_scope(scope_name):

                    conv = tf.nn.conv2d(input = input, filter = conv_filters(filter_shape), strides = strides, padding = 'SAME') 
                    conv = _bias_add_scope(conv, [filter_shape[-1]])
                    conv = tf.contrib.layers.batch_norm(conv, is_training = self.is_training)
                    conv = tf.nn.relu(conv)
                    conv = tf.nn.dropout(conv, self.conv_keep_prob)

                return conv

            def _avg_scope(input, strides, num_conv_filters):
                """Creates a scope around the average-pool path."""
                with tf.variable_scope('average'):
                    avg_pool = tf.nn.avg_pool(value = input, ksize = strides, strides = strides, padding = "SAME")

                    convolution_filter_shape = [1,1,avg_pool.get_shape().as_list()[3], num_conv_filters]
                    avg = _conv_scope(avg_pool, convolution_filter_shape, [1,1,1,1], scope_name = "1x1_conv")

                return avg

            def _max_scope(input, strides,  num_conv_filters):
                """Creates a scope around the max-pool path"""
                with tf.variable_scope('max'):
                    max_pool = tf.nn.max_pool(value = input, ksize = strides, strides = strides, padding = "SAME")

                    convolution_filter_shape = [1,1,max_pool.get_shape().as_list()[3],num_conv_filters]
                    max_ = _conv_scope(max_pool, convolution_filter_shape, [1,1,1,1], scope_name = "1x1_conv")

                return max_

            def _filter_cat_scope(filters):
                """Creates a scope around filter concatation (layer output)"""
                with tf.variable_scope('filter_cat'):
                    filter_cat = tf.concat(filters, 3)
                return filter_cat

            ######

            with tf.variable_scope(layer_name):

                narrow_conv_width = wide_conv_width / 2

                num_narrow_conv_filters = np.max([num_1x1_conv_filters / 2, 2])
                num_wide_conv_filters = np.max([num_narrow_conv_filters / 2, 1])

                _1x1_strides = [1,1,1,1]

                avg_output = _avg_scope(input, strides, num_1x1_conv_filters)
                max_output = _max_scope(input, strides, num_1x1_conv_filters)

                inital_conv = _conv_scope(input, [1,1,input.get_shape().as_list()[3],num_1x1_conv_filters], [1,1,1,1], '1x1_conv')

                narrow_convolution = _conv_scope(inital_conv, [1,narrow_conv_width,inital_conv.get_shape().as_list()[3],num_narrow_conv_filters], strides, scope_name = 'narrow')
                wide_convolution = _conv_scope(inital_conv, [1,wide_conv_width,inital_conv.get_shape().as_list()[3],num_wide_conv_filters], strides, scope_name = 'wide')

                catted_filters = _filter_cat_scope([avg_output, narrow_convolution, wide_convolution, max_output])

            return catted_filters

        # creates the network graph
        tf.reset_default_graph()
        
        self.is_training = tf.placeholder(dtype = tf.bool, name = 'is_training', shape = [])

        with tf.variable_scope('keep_probs'):
            
            self.sample_keep_prob = tf.placeholder(self.dtype, name = 'sample_keep_prob')
            self.conv_keep_prob = tf.placeholder(self.dtype, name = 'conv_keep_prob') 
            
            # keep prob for the layer going into prediction
            self.pred_keep_prob = tf.placeholder(self.dtype, name = 'pred_keep_prob')    



        with tf.variable_scope('sample'):
            # holds the 1 x num_channels samples that are fed into the network
            self.X = tf.placeholder(self.dtype, shape = [None, 1, self.num_freq_channels, 1], name = 'X')
            self.X_dropout = tf.nn.dropout(self.X, self.sample_keep_prob)

        self._layers = []
        num_layers = len(self.wide_filter_widths)
        layer_names = ['layer_{}'.format(i) for i in range(num_layers)]

        for i in range(num_layers):
            # previous layer is input for current layer
            input = self.X_dropout if i == 0 else self._layers[i - 1]
            strides = [1, 1, self.width_reduction_factors[i], 1]
            q_layer = quad_path_layer(input, self.wide_filter_widths[i], strides, layer_names[i], num_1x1_conv_filters = self.num_1x1_conv_filters)
            self._layers.append(q_layer)
            
        with tf.variable_scope('prediction'):
            reshaped_final_layer = tf.contrib.layers.flatten(self._layers[-1])
            reshaped_final_layer = tf.nn.dropout(reshaped_final_layer, self.pred_keep_prob)
            prediction_weight = tf.get_variable(name = 'weight', shape = [reshaped_final_layer.get_shape()[-1], 1], dtype = self.dtype, initializer = tf.contrib.layers.xavier_initializer())
            self.predictions = tf.matmul(reshaped_final_layer, prediction_weight)

        with tf.variable_scope('targets'):
            self.targets = tf.placeholder(dtype = self.dtype, shape = [None, 1], name = 'targets')

        with tf.variable_scope('costs'):

            error = tf.subtract(self.targets, self.predictions, name = 'error')
            squared_error = tf.square(error, name = 'squared_difference')

            with tf.variable_scope('mean_inverse_shifted_gaussian'):
                with tf.variable_scope('normal_distribution'):
                    
                    sigma = tf.constant(self.threshold, name = 'sigma')
                    normal_dist = tf.contrib.distributions.Normal(0.0, sigma, name = 'normal_dist')
                    gaussian_prob = normal_dist.prob(error, name = 'gaussian_prob')
                    shift = tf.constant(self.g_shift, name = 'gaussian_shift_value')
                    shifted_gaussian = tf.add(gaussian_prob, shift, name = 'shifted_gaussian')   
                    
                self.MISG = tf.reduce_mean(tf.divide(1.0, shifted_gaussian), name = 'mean_inverse_shifted_gaussian')
                
            with tf.variable_scope('mean_squared_error'):
                self.MSE = tf.reduce_mean(squared_error)
        
        update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
        with tf.control_dependencies(update_ops):
            with tf.variable_scope('train'):
            
                cost = self.MSE if self.cost_name == 'MSE' else self.MISG
                LR = tf.constant(self.learning_rate, name = 'learning_rate')

                self.optimizer = tf.train.AdamOptimizer(LR, epsilon=1e-08).minimize(cost)

        with tf.variable_scope('logging'):  

            with tf.variable_scope('image'):
                self.image_buf = tf.placeholder(tf.string, shape=[])
                epoch_image = tf.expand_dims(tf.image.decode_png(self.image_buf, channels=4), 0)

            with tf.variable_scope('percent_within_threshold'):
                self.PWT = 100.*tf.reduce_mean(tf.cast(tf.less_equal(tf.abs(self.targets - self.predictions), sigma), self.dtype) )


            tf.summary.histogram(name = 'targets', values = self.targets)
            tf.summary.histogram(name = 'predictions',values =  self.predictions)
            tf.summary.scalar(name = 'MSE', tensor = self.MSE)
            tf.summary.scalar(name = 'MISG', tensor = self.MISG)
            tf.summary.scalar(name = 'PWT', tensor = self.PWT)
            tf.summary.image('prediction_vs_actual', epoch_image)
            self.summary = tf.summary.merge_all()
            
        sys.stdout.write('\rNetwork Ready')
